In [1]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import os
import gzip
import zipfile
import shutil
from time import sleep
import pickle
from time import sleep
import sys
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import clear_output
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text

In [2]:
with open('query_dict/id_querry_spelled.pickle', 'rb') as file:
    id_querry_clean = pickle.load(file)
with open('title_dict/title_data.pickle', 'rb') as file:
    title_data = pickle.load(file)

In [3]:
df_marks = pd.read_csv('train.marks.tsv', delimiter='\t', header=None)
df_marks = df_marks.rename(columns={0: "QueryId", 1: "DocumentId"})
df_marks = df_marks.drop(columns=[2])
df_example = pd.read_csv('sample.csv')
all_groups = df_marks.append(df_example)
all_groups = all_groups.reset_index()
all_groups = all_groups.drop(columns=['index'])
all_groups = all_groups.sort_values(by=['QueryId', 'DocumentId']).reset_index()
all_groups = all_groups.drop(columns=['index'])
rev_frame = all_groups.sort_values(by=['DocumentId', 'QueryId']).reset_index()
rev_frame = rev_frame.drop(columns=['index'])

In [4]:
model1 = hub.load('https://tfhub.dev/google/universal-sentence-encoder-multilingual/3')

In [5]:
neuro_feats = dict()
neuro_feats['use-multilingual/3_unnormed_512_texts'] = []

In [6]:
q_ids = all_groups['QueryId'].unique()
for q1 in tqdm(q_ids):
    corpus = []
    tmp = all_groups[all_groups.QueryId==q1]
    docs = tmp['DocumentId'].values
    for doc1 in docs:
        with gzip.open("doc_collection/{}/{}.gz".format(q1, doc1)) as file:
            for line in file:
                a = line.decode('utf-8', errors = 'ignore').lower().strip('\n').split('\t')
                a = ' '.join(a[0].split()[:512])
        corpus.append((a, doc1))
    cur_q = id_querry_clean[q1]
    one_tok_txt = cur_q.split()
    Q_emb = model1(cur_q)
    for doc1 in corpus:
        D_emb = model1(doc1[0])
        neuro_feats['use-multilingual/3_unnormed_512_texts'].append(cosine_similarity(Q_emb.numpy(), D_emb.numpy()).flatten()[0])

100%|██████████| 6311/6311 [11:35:24<00:00,  6.61s/it]  


In [7]:
with open('use-multilingual_3_unnormed_512_texts.pickle', 'wb') as file:
    pickle.dump(neuro_feats, file)